# The four features' definition 

N -> Press

W -> Release

- HT = W(i-1) - N(i-1)

- PPT = Ni - N(i-1)

- RRT = Wi - W(i-1)

- RPT = Ni - W(i-1)



# Importing dependencies and paths

In [28]:
import os
import pandas as pd
import numpy as np
import joblib
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

path = os.getcwd()
data = pd.read_csv(path + '\Train_keystroke.csv')

# Creating a dataframe

In [29]:
df = pd.DataFrame() 

#  Generating HT, PPT, RRT, RPT for each two consecutive keys

In [30]:

for _, row in data.iterrows():

    list_ht = [row[2] - row[1]]
    list_ppt = []
    list_rrt = []
    list_rpt = []

    for i in range(4, len(row), 2):
        ht = row[i] - row[i-1]
        ppt = row[i-1] - row[i-3]
        rrt = row[i] - row[i-2]
        rpt = row[i-1] - row[i-2]

        list_ht.append(ht)
        list_ppt.append(ppt)
        list_rrt.append(rrt)
        list_rpt.append(rpt)

    mean_ht = np.mean(list_ht)
    mean_ppt = np.mean(list_ppt)
    mean_rrt = np.mean(list_rrt)
    mean_rpt = np.mean(list_rpt)

    std_ht = np.std(list_ht)
    std_ppt = np.std(list_ppt)
    std_rrt = np.std(list_rrt)
    std_rpt = np.std(list_rpt)

                  
    data = {'HT.Mean': mean_ht, 'HT.STD': std_ht,
            'PPT.Mean': mean_ppt, 'PPT.STD': std_ppt,
            'RRT.Mean': mean_rrt, 'RRT.STD': std_rrt,
            'RPT.Mean': mean_rpt, 'RPT.STD': std_rpt,
            'UserID': row[0]}
            
    new_df =  pd.DataFrame(pd.Series(data), index=None).T
    new_df.reset_index(drop=True, inplace=True)
    df = pd.concat([df, new_df], ignore_index=True,  axis=0)

# Spliting dataset into training set and test set

In [31]:
y = df['UserID']
X = df.drop(['UserID'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3) # 70% training and 30% test

# Creating SVM Classifier and saving model

In [32]:

svc = SVC()
svc_classifier = OneVsRestClassifier(svc).fit(X_train, y_train)
y_pred = svc_classifier.predict(X_test)
joblib.dump(svc_classifier, 'svm_model.sav')

['svm_model.sav']

# Creating Random Forest Classifier and saving model

In [33]:

rf_classifier = RandomForestClassifier().fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
joblib.dump(rf_classifier, 'rf_model.sav')

['rf_model.sav']

# Creating XGBoost Classifier and saving model

In [34]:

xgb_classifier = OneVsRestClassifier(XGBClassifier()).fit(X, y)
y_pred = xgb_classifier.predict(X_test)
joblib.dump(xgb_classifier, 'xgb_model.sav')

['xgb_model.sav']